In [12]:
import imageio
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader

In [20]:
def preprocess_imagery(path):
    '''
    Pre process satellite imagery. We check for nans in the composite images,
    and then we load them into the [0,1] range.
    Input: TIF files downloaded from GEE (224x224x7)
    '''
    #reading image
    image=imageio.imread(path)
    if np.sum(np.isnan(image))>0:
        #print(str(np.sum(np.isnan(image)))+"nans found")
        #taking the mean of the image, per channel
        mean=np.nanmean(image,axis=(0,1))
        #replacing NaN with per channel mean
        replacement=np.isnan(image)*mean
        image[np.isnan(image)]=replacement[np.isnan(image)]
    image = np.transpose(image, (1, 2, 0))    
    return image

In [21]:
image_files = [f for f in os.listdir("../GEE_Images")]
image_info=[]
image_files=image_files[:5000]
len(image_files)

5000

In [22]:
for image in image_files:
    name,_=os.path.splitext(image)
    image_info.append(['_'.join(name.split(' ')[0].split('_')[:-1]),name.split(' ')[0].split('_')[-1]])

In [23]:
df=pd.read_csv('processed_data.csv')
mp10=[]

In [24]:
for info in image_info:
    mp10.append(df[(df['station']==info[0]) & (df['Date']==info[1])]['mp10'].values[0])

In [25]:
images=[]

In [26]:
for file in image_files:
    image = preprocess_imagery("../GEE_Images/"+file)
    images.append(transforms.ToTensor()(image))

C:\Users\benja\AppData\Local\Temp\ipykernel_16520\2215959288.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image=imageio.imread(path)


In [27]:
X_train, X_test, y_train, y_test = train_test_split(images, mp10, test_size=0.2, random_state=42)

In [31]:
np.shape(images[0])

torch.Size([224, 224, 6])

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Definimos el modelo
model = Sequential()

# Agregamos las capas convolucionales y de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224,224,6)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Aplanamos el tensor
model.add(Flatten())

# Agregamos las capas densas
model.add(Dense(128, activation='relu'))
model.add(Dense(1))  # Una sola neurona de salida para la predicción del valor float

# Compilamos el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Mostramos un resumen del modelo
model.summary()


c:\Users\benja\anaconda3\envs\ee\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 32)   │         1,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,953 (42.61 MB)

 Trainable params: 11,169,953 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
np.shape(y_train)

torch.Size([224, 224, 6])

In [39]:
# Entrenamos el modelo
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=5)


In [ ]:
# Evaluamos el modelo
test_loss = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
